In [18]:
import sagemaker as sg
from pyspark import sql

In [4]:
import boto3

In [5]:
runtime = boto3.client("runtime.sagemaker")

In [7]:
endpoint = sg.predictor.Predictor("dev-bztext-dbpedia")

In [11]:
endpoint.predict(JSON.dump({"instances": ["words for testing"]}))

NameError: name 'JSON' is not defined